# AG2: GroupChat (Overview)

Overview
- https://docs.ag2.ai/docs/user-guide/advanced-concepts/groupchat/groupchat

## SETUP

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables (for API key)
load_dotenv()

# Set up OpenAI API key
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("Please set the OPENAI_API_KEY environment variable or add it to a .env file")

# Define the model to use
MODEL_GPT = "gpt-4o-mini"

## GroupChat

In [2]:
# from autogen import ConversableAgent, GroupChat, GroupChatManager, LLMConfig
from autogen import ConversableAgent, GroupChat, GroupChatManager

# Put your key in the OPENAI_API_KEY environment variable
# llm_config = LLMConfig(api_type="openai", model="gpt-4o-mini")
llm_config = {"api_type": "openai", "model": "gpt-4o-mini", "api_key": os.environ["OPENAI_API_KEY"]}

planner_message = """You are a classroom lesson agent.
Given a topic, write a lesson plan for a fourth grade class.
Use the following format:
<title>Lesson plan title</title>
<learning_objectives>Key learning objectives</learning_objectives>
<script>How to introduce the topic to the kids</script>
"""

# 1. Add a separate 'description' for our planner and reviewer agents
planner_description = "Creates or revises lesson plans."

lesson_planner = ConversableAgent(
    name="planner_agent",
    llm_config=llm_config,
    system_message=planner_message,
    description=planner_description,
)

reviewer_message = """You are a classroom lesson reviewer.
You compare the lesson plan to the fourth grade curriculum and provide a maximum of 3 recommended changes.
Provide only one round of reviews to a lesson plan.
"""

reviewer_description = """Provides one round of reviews to a lesson plan
for the lesson_planner to revise."""

lesson_reviewer = ConversableAgent(
    name="reviewer_agent",
    llm_config=llm_config,
    system_message=reviewer_message,
    description=reviewer_description,
)

# 2. The teacher's system message can also be used as a description, so we don't define it
teacher_message = """You are a classroom teacher.
You decide topics for lessons and work with a lesson planner.
and reviewer to create and finalise lesson plans.
When you are happy with a lesson plan, output "DONE!".
"""

teacher = ConversableAgent(
    name="teacher_agent",
    llm_config=llm_config,
    system_message=teacher_message,
    # 3. Our teacher can end the conversation by saying DONE!
    is_termination_msg=lambda x: "DONE!" in (x.get("content", "") or "").upper(),
)

# 4. Create the GroupChat with agents and selection method
groupchat = GroupChat(
    agents=[teacher, lesson_planner, lesson_reviewer],
    speaker_selection_method="auto",
    messages=[],
)

# 5. Our GroupChatManager will manage the conversation and uses an LLM to select the next agent
manager = GroupChatManager(
    name="group_manager",
    groupchat=groupchat,
    llm_config=llm_config,
)

In [3]:
# 6. Initiate the chat with the GroupChatManager as the recipient
chat_result = teacher.initiate_chat(
    recipient=manager,
    message="Today, let's introduce our kids to the solar system."
)

teacher_agent (to group_manager):

Today, let's introduce our kids to the solar system.

--------------------------------------------------------------------------------

Next speaker: planner_agent


>>>>>>>> USING AUTO REPLY...
planner_agent (to group_manager):

<title>Exploring the Solar System</title>
<learning_objectives>
1. Students will be able to identify and name the planets in our solar system.
2. Students will understand the basic characteristics of each planet.
3. Students will create a simple model of the solar system to demonstrate their understanding.
</learning_objectives>
<script>
"Good morning, class! Today we’re going to embark on an exciting journey through our solar system! Have you ever looked up at the night sky and wondered about the stars and planets? Today, we’ll learn about the amazing planets that revolve around our sun. 

Let’s start with a few questions to get everyone thinking: Can anyone name one of the planets in our solar system? How about the sun — wh

Please give feedback to group_manager. Press enter or type 'exit' to stop the conversation:  exit


In [4]:
print(chat_result.summary)

DONE!


In [5]:
print(len(chat_result.chat_history))
# print(chat_result.chat_history[0])
# print(chat_result.chat_history[-1])
print(chat_result.chat_history)

5
[{'content': "Today, let's introduce our kids to the solar system.", 'role': 'assistant', 'name': 'teacher_agent'}, {'content': '<title>Exploring the Solar System</title>\n<learning_objectives>\n1. Students will be able to identify and name the planets in our solar system.\n2. Students will understand the basic characteristics of each planet.\n3. Students will create a simple model of the solar system to demonstrate their understanding.\n</learning_objectives>\n<script>\n"Good morning, class! Today we’re going to embark on an exciting journey through our solar system! Have you ever looked up at the night sky and wondered about the stars and planets? Today, we’ll learn about the amazing planets that revolve around our sun. \n\nLet’s start with a few questions to get everyone thinking: Can anyone name one of the planets in our solar system? How about the sun — what do you think it is? Great answers! \n\nNow, I want you all to imagine what it must be like living on a different planet. W